# Bokeh Intro

![bokeh_logo](../images/bokeh_logo.png)

Bokeh was one of the first projects to implement the "JavaScript rendering from Python" idea, creating their own BokehJS library to handle the plotting. Bokeh was started by Anaconda and is fairly popular in the Data Science community as a result. Bokeh has a longer production history, and libraries like Dask use it to run their Dashboard Visualizations. It's definitely one of the more mature of the new breed.

Let's get a feel for it with some basic plots by generating some numpy data

In [1]:
from bokeh.plotting import figure, output_notebook
from bokeh.io import show
from bokeh.layouts import layout
import numpy as np

x = np.arange(1, 50)
y = np.sin(x)

Bokeh has had Notebook support from the beginning, just run one line and Bokeh sets up the JavaScript stuff needed to render the graphs

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
p = figure()
p.line(x, y)
show(p)

Out of the box, we get a fair amount of interactivity - we are able to pan and zoom.

The API also seems very matplotlib OOPish

In [4]:
p2 = figure()
p2.vbar(x=x, width=0.8, top=y)
show(p2)

We can of course layout the graps together

In [5]:
show(layout([[p, p2]]))

We can add some simple interactivity by setting properties of the graphs equal to each other

In [6]:
p.x_range = p2.x_range

In [7]:
show(layout([[p, p2]]))

There are two main sources of interactivity in Bokeh, sharing datasources and using the bokeh server. We can share datasources by using a ColumnDataSource, which can be built from a dataframe automatically. Let's build the same petal vs sepal comparison as before.

By specifiying a source, we can just pass the name of the column we want to use. We can also specify what tools we want to be able to use in our graph

In [8]:
from bokeh.models import ColumnDataSource
from bokeh.palettes import Colorblind3
from bokeh.transform import factor_cmap, CategoricalColorMapper
from vega_datasets import data

In [9]:
iris = data.iris()

In [10]:
source = ColumnDataSource(iris)
tools = "box_select,lasso_select,help"

In [11]:
petals = figure()
petals.circle(x='petalLength', y='petalWidth', color=factor_cmap('species', Colorblind3, iris.species.unique()), source=source)
sepal = figure(tools=tools)
sepal.circle(x='sepalLength', y='sepalWidth', color=factor_cmap('species', Colorblind3, iris.species.unique()), source=source)

show(layout([[petals, sepal]]))

Just by sharing the same datasource, the graphs are now linked!

A big part of Bokeh is the ability to run a bokeh server, which let's you setup callbacks between graphs and widgets - Bokeh has a ton of widgets set up

In [12]:
from bokeh.models import Slider, Button, RangeSlider, Select, RadioButtonGroup
from bokeh.layouts import column

In [13]:
slider = Slider(start=0, end=10, value=1, step=.1, title="Slider")
button_group = RadioButtonGroup(labels=["Option 1", "Option 2", "Option 3"], active=0)
select = Select(title="Option:", value="foo", options=["foo", "bar", "baz", "quux"])
button_1 = Button(label="Button 1")
button_2 = Button(label="Button 2")

show(column(button_1, slider, button_group, select, button_2, width=300))

Unfortunately, if you want to use them in a notebook, you have to use JavaScript callbacks,using the CustomJS class 

In [14]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = [x*0.005 for x in range(0, 201)]

source = ColumnDataSource(data=dict(x=x, y=x))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    x = data['x']
    y = data['y']
    for (i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)


show(column(slider, plot))

Bokeh has a bokeh server, that lets you attach python callbacks to widgets instead of javascript - which is much nicer, as you can imagine!